In [0]:
# reading from Bronz table and data transformation

In [0]:

import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col

In [0]:
df = spark.table ("workspace.bronze.crm_cust_info")

In [0]:
df.display()

Trimming

In [0]:
# remove the white spaces from the string columns
for field in df.schema.fields:
  if isinstance(field.dataType, StringType):
      df = df.withColumn(field.name, trim(col(field.name)))
df.show()


Normalization


In [0]:
# change the married status and gender abbreviations to full words.
df = (
    df
    .withColumn(
        "cst_marital_status",
        F.when(F.upper(F.col("cst_marital_status")) == "S", "Single")
         .when(F.upper(F.col("cst_marital_status")) == "M", "Married")
         .otherwise("n/a")
    )
    .withColumn(
        "cst_gndr",
        F.when(F.upper(F.col("cst_gndr")) == "F", "Female")
         .when(F.upper(F.col("cst_gndr")) == "M", "Male")
         .otherwise("n/a")
    )
)
df.show()

                

In [0]:
%sql 
select * from workspace.bronze.crm_cust_info
where cst_id is null;

Filter Null Customer Id from table 


In [0]:
df = df.filter(col("cst_id").isNotNull())
df.show()

rename the columns 

In [0]:
#  crate a dictionary which will have the old and new values for the column names and iterate over the dictionary
RENAME_MAP = {
    "cst_id": "customer_id",
    "cst_key": "customer_number",
    "cst_firstname": "first_name",
    "cst_lastname": "last_name",
    "cst_marital_status": "marital_status",
    "cst_gndr": "gender",
    "cst_create_date": "created_date"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)
df.show(5)


Writing to Silver Table 


In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.crm_customers")

In [0]:
%sql 
select * from workspace.silver.crm_customers